In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd drive/MyDrive/work/korean-hate-speech-detection
%ls

/content/drive/MyDrive/work/korean-hate-speech-detection
'=0.20.1'               epoch_24_results.csv   modelsave7/
 data/                  epoch_2_results.csv    my_autotrain_llm/
 dev.hate1.csv          epoch_3_results.csv    offensive.csv
 dev.hate.csv           epoch_4_results.csv    offensive_data_set.csv
 dev.news_title.txt     epoch_5_results.csv    offensive_directory/
 epoch_10_results.csv   epoch_6_results.csv    offensivetest/
 epoch_11_results.csv   epoch_7_results.csv    ourfuckingmodel/
 epoch_12_results.csv   epoch_8_results.csv    pretrained_model_directory/
 epoch_13_results.csv   epoch_9_results.csv    printprob.py
 epoch_14_results.csv   inferenced/            real_final/
 epoch_15_results.csv   install_logs.txt       results/
 epoch_16_results.csv   llama/                 setup_logs.txt
 epoch_17_results.csv   llama2-hoxy/           test.hate.no_label.csv
 epoch_18_results.csv   llamatest.py           test.news_title.txt
 epoch_19_results.csv   modelsave1/          

In [3]:
%pip install transformers

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import pandas as pd
from torch import sigmoid
import os

device = torch.device("cuda:0")

In [ ]:
# 데이터 로드
#hate_data_dev_set_path = (
 #   "hate_data_set.csv"
#)
#hate_data_train_set_path = (
#    "train.hate.csv"
#)

In [5]:
train_data = pd.read_csv("train.hate.csv")
dev_data = pd.read_csv("dev.hate.csv")
train_data['label'].replace("hate", 1, inplace=True)
train_data['label'].replace("none", 0, inplace=True)
train_data['label'].replace("offensive", 0.5, inplace=True)
train_data = train_data[train_data['label'] != 0.5]
dev_data['label'].replace("hate", 1, inplace=True)
dev_data['label'].replace("none", 0, inplace=True)
dev_data['label'].replace("offensive", 0.5, inplace=True)
dev_data = dev_data[dev_data['label'] != 0.5]

In [6]:
# BERT 다국어 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# 문장을 토큰화하고 패딩 적용
tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in train_data["comments"]
]

max_length = max(len(tokens) for tokens in tokenized_texts)
padded_texts = [tokens + [0] * (max_length - len(tokens)) for tokens in tokenized_texts]

# 텐서로 변환
input_ids = (torch.tensor(padded_texts)).to(device)
labels = (torch.tensor(train_data["label"].tolist())).to(device)

val_tokenized_texts = [
    tokenizer.encode(text, add_special_tokens=True) for text in dev_data["comments"]
]
max_length_val = max(len(tokens) for tokens in val_tokenized_texts)
val_padded_texts = [
    tokens + [0] * (max_length_val - len(tokens)) for tokens in val_tokenized_texts
]
val_input_ids = (torch.tensor(val_padded_texts)).to(device)
val_labels = (torch.tensor(dev_data["label"].tolist())).to(device)


# DataLoader 생성
dataset = TensorDataset(input_ids, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
val_dataset = TensorDataset(val_input_ids, val_labels)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)



tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [7]:
# 모델 초기화
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=1
)

model.to(device)

# 옵티마이저 및 손실 함수 초기화
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.BCEWithLogitsLoss()


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# 학습 및 평가
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    total_loss = 0.0
    total_batches = len(data_loader)
    for batch_idx, batch in enumerate(data_loader):
        input_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids)
        logits = outputs.logits.squeeze(1)
        loss = criterion(logits, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # 출력 형식: Epoch [현재 에폭], Batch [현재 배치 / 전체 배치 수], Loss: [현재 배치 손실]
        print(
            f"Epoch {epoch + 1}, Batch {batch_idx + 1} / {total_batches}, Loss: {loss.item()}"
        )

    # 평가
    model.eval()
    with torch.no_grad():
        predictions = []
        true_labels = []
        probabilities_hate = []
        probabilities_clean = []
        probabilities_offensive = []

        for batch in data_loader:
            input_ids, labels = batch
            outputs = model(input_ids)
            logits = outputs.logits.squeeze(1)
            prob_hate = sigmoid(logits)
            prob_clean = 1 - prob_hate
            prob_offensive = prob_hate
            predictions.extend((prob_hate > 0.5).float().tolist())  # 0.5를 기준으로 이진 분류
            true_labels.extend(labels.tolist())
            probabilities_hate.extend(prob_hate.tolist())
            probabilities_clean.extend(prob_clean.tolist())
            probabilities_offensive.extend(prob_offensive.tolist())



    accuracy = accuracy_score(true_labels, predictions)
    auc_roc = roc_auc_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)

    print(
        f"Epoch {epoch + 1}, Loss: {total_loss / len(data_loader)}, Accuracy: {accuracy}, Auc: {auc_roc}, f1: {f1}"
    )

    model.eval()
    with torch.no_grad():
        val_predictions = []
        val_true_labels = []
        for val_batch in val_loader:
            val_input_ids, val_labels = val_batch
            val_outputs = model(val_input_ids)
            val_logits = val_outputs.logits.squeeze(1)
            val_predictions.extend(torch.sigmoid(val_logits).round().tolist())
            val_true_labels.extend(val_labels.tolist())

        val_accuracy = accuracy_score(val_true_labels, val_predictions)
        print(
            f"Epoch {epoch + 1}, Train Loss: {total_loss / len(val_loader)}, Validation Accuracy: {val_accuracy}"
        )

    # # 각 문장에 대한 확률값 및 실제 라벨 출력
    # for text, prob_hate, prob_clean, prob_offensive, true_label in zip(
    #     dev_data["comments"],
    #     probabilities_hate,
    #     probabilities_clean,
    #     probabilities_offensive,
    #     dev_data["Value"],
    # ):
    #     if true_label == 1:
    #         print(f"Text: {text}, True Label: Hate, Probability for Hate: {prob_hate}")
    #     elif true_label == 0:
    #         print(
    #             f"Text: {text}, True Label: Clean, Probability for Clean: {prob_clean}"
    #         )



We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 2, Batch 409 / 1350, Loss: 0.2412426769733429
Epoch 2, Batch 410 / 1350, Loss: 0.4944400191307068
Epoch 2, Batch 411 / 1350, Loss: 0.1728072166442871
Epoch 2, Batch 412 / 1350, Loss: 0.2151697874069214
Epoch 2, Batch 413 / 1350, Loss: 0.6543656587600708
Epoch 2, Batch 414 / 1350, Loss: 0.35471153259277344
Epoch 2, Batch 415 / 1350, Loss: 0.3402089476585388
Epoch 2, Batch 416 / 1350, Loss: 0.1543782353401184
Epoch 2, Batch 417 / 1350, Loss: 0.20169471204280853
Epoch 2, Batch 418 / 1350, Loss: 0.3105595111846924
Epoch 2, Batch 419 / 1350, Loss: 0.27242714166641235
Epoch 2, Batch 420 / 1350, Loss: 0.15257857739925385
Epoch 2, Batch 421 / 1350, Loss: 0.6416348814964294
Epoch 2, Batch 422 / 1350, Loss: 0.14445023238658905
Epoch 2, Batch 423 / 1350, Loss: 0.16020822525024414
Epoch 2, Batch 424 / 1350, Loss: 0.1191558763384819
Epoch 2, Batch 425 / 1350, Loss: 0.09626886248588562
Epoch 2, Batch 426 / 1350, Loss: 0.2684706151485443
Epoch 2, Batch 427 / 

In [ ]:

# 모델 저장
model.save_pretrained("pretrained_model_directory")  # type: ignore
